In [ ]:
import pandas as pd 
from datetime import datetime, timedelta
import os

airports = ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']

# Set up master dataframe
start_date = '2022-09-01 00:00:00'
end_date = '2023-08-31 23:59:59'
time_index = pd.date_range(start=start_date, end=end_date, freq='15min', inclusive='both')

master_df = pd.DataFrame(index=time_index)
master_df.index.name = 'prediction_time'

pd.set_option('display.max_columns', None)
master_df.head()

In [14]:
import os 

fuser_train = 'data/FUSER/FUSER_train'
fuser_test = 'data/FUSER/FUSER_test'

cwam_train = 'data/CWAM/CWAM_train'
cwam_test = 'data/CWAM/CWAM_test'

metar_train = 'data/METAR/METAR_train'
metar_test = 'data/METAR/METAR_test'

taf_train ='data/TAF/TAF_train'
taf_test = 'data/TAF/TAF_test'

data_out = 'cleaned_data'

In [ ]:
def get_fuser_files(pattern, airport):
    files = []
    for split in [fuser_train, fuser_test]:
        for filename in os.listdir(os.path.join(split, airport)):
            if len(filename) < len('KATL_2022-09-01_2022-10-02.runways_data_set.csv') and pattern in filename:
                files.append(os.path.join(split, airport, filename))
    return files

def process_runways_data(airport):
    files = get_fuser_files('runways_data_set', airport)
    
    # Concatenate all files for this airport into one combined_df
    combined_df = pd.concat(
        [pd.read_csv(f, parse_dates=['arrival_runway_actual_time', 'departure_runway_actual_time']) for f in files],
        ignore_index=True
    )
    
    # Process arrivals
    arrivals_df = (
        combined_df[['gufi', 'arrival_runway_actual_time']]
        .dropna()
        .sort_values('arrival_runway_actual_time')
        .drop_duplicates('gufi', keep='last')
    )
    
    # Process departures
    departures_df = (
        combined_df[['gufi', 'departure_runway_actual_time']]
        .dropna()
        .sort_values('departure_runway_actual_time')
        .drop_duplicates('gufi', keep='last')
    )
    
    # Convert to time series
    arrivals = pd.Series(1, index=arrivals_df['arrival_runway_actual_time'])
    departures = pd.Series(1, index=departures_df['departure_runway_actual_time'])
    
    # Resample to 15-min intervals and count
    arrival_counts = arrivals.resample('15min').count().rename(f'actual_arrivals_{airport}')
    departure_counts = departures.resample('15min').count().rename(f'actual_departures_{airport}')
    
    counts_df = pd.concat([arrival_counts, departure_counts], axis=1).fillna(0)
    
    return counts_df

    
actual_arrivals_departures_df = pd.concat([process_runways_data(airport) for airport in airports], axis=1)

In [ ]:
actual_arrivals_departures_df.head()

In [ ]:
def process_tbfm_data(airport):
    print('Processing TBFM for ' + airport)
    files = get_fuser_files('TBFM', airport)
    
    # Concatenate all files for this airport into one combined_df
    combined_df = pd.concat(
        [pd.read_csv(f, parse_dates=['arrival_runway_sta']) for f in files],
        ignore_index=True
    )
    
    # Process arrivals
    arrivals_df = (
        combined_df[['gufi', 'arrival_runway_sta']]
        .dropna(subset=['arrival_runway_sta'])
        .drop_duplicates(subset='gufi', keep='last')
        .reset_index(drop=True)
    )
    
    # Convert to time series
    arrivals = pd.Series(1, index=arrivals_df['arrival_runway_sta'])
    
    # Resample to 15-minute intervals and count
    arrival_counts = arrivals.resample('15min').count().rename(f'scheduled_arrivals_{airport}')
    
    # Ensure the index is sorted before concatenation
    arrival_counts = arrival_counts.sort_index()
    counts_df = arrival_counts.to_frame().fillna(0)
    
    return counts_df
    
scheduled_arrivals_df = pd.concat([process_tbfm_data(airport) for airport in airports], axis=1)    


In [ ]:
scheduled_arrivals_df.head()

In [ ]:

def process_tfm_data(airport):
    print('Processing TFM for ' + airport)
    files = get_fuser_files('TFM', airport)
    
    # Concatenate all files for this airport into one combined_df
    combined_df = pd.concat(
        [pd.read_csv(f, parse_dates=['arrival_runway_estimated_time']) for f in files],
        ignore_index=True
    )
    
    # Process arrivals
    arrivals_df = (
        combined_df[['gufi', 'arrival_runway_estimated_time']]
        .dropna(subset=['arrival_runway_estimated_time'])
        .drop_duplicates(subset='gufi', keep='last')
        .reset_index(drop=True)
    )
    
    # Convert to time series
    arrivals = pd.Series(1, index=arrivals_df['arrival_runway_estimated_time'])
    
    # Resample to 15-minute intervals and count
    arrival_counts = arrivals.resample('15min').count().rename(f'estimated_arrivals_{airport}')
    
    arrival_counts = arrival_counts.sort_index()
    counts_df = arrival_counts.to_frame().fillna(0)
    
    return counts_df
    
estimated_arrivals_df = pd.concat([process_tfm_data(airport) for airport in airports], axis=1)    
    
    


In [ ]:
estimated_arrivals_df.head(10)

In [ ]:
def process_etd_data(airport):
    print('processing ETD for ' + airport)
    files = get_fuser_files('ETD', airport)
    
    # Concatenate all files for this airport into one combined_df
    combined_df = pd.concat(
        [pd.read_csv(f, parse_dates=['departure_runway_estimated_time']) for f in files],
        ignore_index=True
    )
    
    # Process departuress
    departures_df = (
        combined_df[['gufi', 'departure_runway_estimated_time']]
        .dropna()
        .drop_duplicates(subset='gufi', keep='last')
        .reset_index(drop=True)
    )
    
    # Convert to time series
    departures = pd.Series(1, index=departures_df['departure_runway_estimated_time'])
    
    # Resample to 15-minute intervals and count
    departure_counts = departures.resample('15min').count().rename(f'estimated_departures_{airport}')
    
    departure_counts = departure_counts.sort_index()
    counts_df = departure_counts.to_frame().fillna(0)
    
    return counts_df

estimated_departures_df = pd.concat([process_etd_data(airport) for airport in airports], axis=1)



In [ ]:
estimated_departures_df.head(250)

In [ ]:
def process_lamp_data(airport):
    print('processing lamp data for ' + airport)
    files = get_fuser_files('LAMP', airport)
    
    # Concatenate all files for this airport into one combined_df
    combined_df = pd.concat(
        [pd.read_csv(f, parse_dates=['forecast_timestamp']) for f in files],
        ignore_index=True
    )
    
    # Convert categorical columns
    combined_df['cloud'] = pd.Categorical(combined_df['cloud']).codes
    combined_df['lightning_prob'] = pd.Categorical(combined_df['lightning_prob']).codes
    combined_df['precip'] = combined_df['precip'].astype(int)
    
    # Process featuers
    columns_to_process = [
        'temperature',
        'wind_direction',
        'wind_speed',
        'wind_gust',
        'cloud_ceiling',
        'lightning_prob',
        'precip'
    ]
    
    processed_dfs = []
    
    for col in columns_to_process:
        temp_df = combined_df[['forecast_timestamp', col]].drop_duplicates('forecast_timestamp', keep='last')
        temp_df = temp_df.set_index('forecast_timestamp').sort_index()
        
        temp_df[col] = temp_df[col].astype(float)
        
        # Interpolate missing values
        temp_df[col] = temp_df[col].interpolate(method='nearest')
        
        temp_df = temp_df.resample('15min').asfreq()
        
        temp_df[col] = temp_df[col].interpolate(method='nearest')
        processed_dfs.append(temp_df[col].rename(f'{col}_{airport}'))
      
    if not processed_dfs:
        return pd.DataFrame()
    
    # Concatenate all processed series
    counts_df = pd.concat(processed_dfs, axis=1)
    
    return counts_df


simple_weather_df = pd.concat([process_lamp_data(airport) for airport in airports], axis=1)


In [ ]:
simple_weather_df.head(10)

In [14]:
all_dfs = [master_df, actual_arrivals_departures_df, scheduled_arrivals_df, estimated_arrivals_df, estimated_departures_df, simple_weather_df]

# master_df = pd.concat(all_dfs, axis=1, join='outer')
master_df = master_df.join(all_dfs, how = 'left')
master_df = master_df.fillna(0)
master_df = master_df[sorted(master_df.columns)]

master_df.to_csv('all_fuser.csv')

In [ ]:
from metar import Metar

# Gather file paths
files = []
for file in os.listdir(metar_train):
    filepath = os.path.join(metar_train, file)
    files.append(filepath)

for file in os.listdir(metar_test):
    filepath = os.path.join(metar_test, file)
    files.append(filepath)

files.sort(key=lambda x: x[-17:])

def format_weather_condition(weather_condition):
    if isinstance(weather_condition, str):
        return weather_condition
    elif isinstance(weather_condition, tuple):
        items = [str(item) for item in weather_condition if item is not None]
        return ' '.join(items)
    else:
        return str(weather_condition)

def format_sky_condition(sky_condition):
    # Attempt to extract cover, height, cloud_type
    if isinstance(sky_condition, tuple):
        cover = sky_condition[0] if len(sky_condition) > 0 else None
        height = sky_condition[1] if len(sky_condition) > 1 else None
        cloud_type = sky_condition[2] if len(sky_condition) > 2 else None
    else:
        cover = getattr(sky_condition, 'cover', None)
        height = getattr(sky_condition, 'height', None)
        cloud_type = getattr(sky_condition, 'cloud_type', None)

    parts = []
    if cover:
        parts.append(cover)
    if height:
        parts.append(str(height))
    if cloud_type:
        parts.append(cloud_type)
    return ' '.join(parts)

def parse_metar(metar_line, year, month):
    try:
        obs = Metar.Metar(metar_line, year=year, month=month, strict=False)
    except Metar.ParserError as e:
        print(f"Error parsing METAR line: {metar_line}\n{e}")
        return None
    except Exception as e:
        print(f"Unexpected error parsing METAR line: {metar_line}\n{e}")
        return None

    data = {
        'station_id': obs.station_id,
        'temp_c': obs.temp.value('C') if obs.temp else None,
        'dewpt_c': obs.dewpt.value('C') if obs.dewpt else None,
        'wind_dir_degrees': obs.wind_dir.value() if obs.wind_dir else None,
        'wind_speed_kt': obs.wind_speed.value() if obs.wind_speed else None,
        'wind_gust_kt': obs.wind_gust.value() if obs.wind_gust else None,
        'visibility_m': obs.vis.value('M') if obs.vis else None,
        'altimeter_hpa': obs.press.value('HPA') if obs.press else None,
        'weather': ' '.join(format_weather_condition(w) for w in obs.weather) if obs.weather else '',
        'clouds': ' '.join(format_sky_condition(c) for c in obs.sky) if obs.sky else '',
    }
    return data

def process_metar_data():
    all_data = []

    for idx, filepath in enumerate(files):
        try:
            with open(filepath, 'r', encoding='ISO-8859-1', errors='replace') as f:
                lines = f.readlines()

            # Process lines in triplets: date line, metar line, blank line
            for i in range(0, len(lines), 3):
                if i + 1 >= len(lines):
                    break
                date_time_line = lines[i].strip()
                metar_line = lines[i+1].strip()

                if not metar_line:
                    continue

                first_token = metar_line.split(' ')[0] if metar_line else ''
                if first_token not in airports:
                    continue

                # Parse observation_time
                try:
                    observation_time = datetime.strptime(date_time_line, "%Y/%m/%d %H:%M")
                except ValueError:
                    continue  # skip if datetime can't be parsed

                # Parse the METAR line
                year = int(date_time_line[:4])
                month = int(date_time_line[5:7])
                metar_data = parse_metar(metar_line, year, month)
                if metar_data is None:
                    continue

                metar_data['observation_time'] = observation_time
                all_data.append(metar_data)

        except UnicodeDecodeError as e:
            print(f"Error reading file {filepath}: {e}")
        except Exception as e:
            print(f"Unexpected error processing file {filepath}: {e}")

    if not all_data:
        print("No METAR data processed.")
        return pd.DataFrame()

    combined_df = pd.DataFrame(all_data)
    combined_df.set_index('observation_time', inplace=True)

    # Remove duplicates
    combined_df = combined_df[~combined_df.index.duplicated(keep='last')]

    # Factorize categorical columns
    combined_df['weather_code'], _ = pd.factorize(combined_df['weather'])
    combined_df['clouds_code'], _ = pd.factorize(combined_df['clouds'])
    combined_df.drop(['weather', 'clouds'], axis=1, inplace=True)

    # Pivot so each feature is separated by airport
    combined_df.reset_index(inplace=True)
    pivoted = combined_df.pivot(index='observation_time', columns='station_id')

    # Flatten columns
    pivoted.columns = [f"{feat}_{stn}" for feat, stn in pivoted.columns]
    pivoted.index = pd.to_datetime(pivoted.index)
    pivoted.sort_index(inplace=True)

    # Identify numeric and categorical code columns
    categorical_cols = [col for col in pivoted.columns if 'weather_code' in col or 'clouds_code' in col]
    numeric_cols = [col for col in pivoted.columns if col not in categorical_cols]

    # Handle numeric data: Resample and interpolate
    numeric_data = pivoted[numeric_cols].resample('15min').mean().interpolate(method='time')

    categorical_data = pivoted[categorical_cols].resample('15min').ffill()

    # Combine numeric and categorical
    resampled = pd.concat([numeric_data, categorical_data], axis=1)

    # For categorical codes, ensure they are integers. If NaN appears, fill with -1.
    for col in categorical_cols:
        resampled[col] = resampled[col].fillna(-1).astype(int)

    return resampled

metar_data = process_metar_data()



In [ ]:
#metar_data.columns = ['metar_'+ col for col in metar_data.columns]
metar_data = metar_data[sorted(metar_data.columns)]

metar_data.head()

In [34]:
master_df = master_df.join(metar_data, how = 'left')
master_df = master_df.fillna(0)

master_df.to_csv('fuser_and_metar.csv')

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta
import re

# Gather file paths
files = []
for file in os.listdir(taf_train):
    filepath = os.path.join(taf_train, file)
    files.append(filepath)

for file in os.listdir(taf_test):
    filepath = os.path.join(taf_test, file)
    files.append(filepath)

files.sort(key=lambda x: x[-17:])

print(f"Found {len(files)} TAF files to process")


def parse_taf_line(line):
    """Parse a single TAF line to extract relevant weather information."""
    # Initialize default values
    data = {
        'station': None,
        'wind_dir': None, 
        'wind_speed': None,
        'wind_gust': None,
        'visibility': None,
        'clouds': [],
        'weather': None,
        'valid_from': None,
        'valid_to': None
    }
    
    # Extract station ID (first 4 characters after 'TAF')
    if 'TAF' in line:
        parts = line.split()
        try:
            station_idx = parts.index('TAF') + 1
            if station_idx < len(parts):
                data['station'] = parts[station_idx][:4]
        except ValueError:
            # Handle case where 'TAF' is in line but not as a separate word
            pass
    else:
        # Try to get first 4-letter code
        match = re.search(r'\b[A-Z]{4}\b', line)
        if match:
            data['station'] = match.group()
    
    # Check if the extracted station is in the predefined list of airports
    if data['station'] not in airports:
        return None  # Skip this line if the station is not in the list
    
    # Extract date/time validity (format like 0106/0206)
    validity = re.search(r'(\d{4})/(\d{4})', line)
    if validity:
        data['valid_from'] = validity.group(1)
        data['valid_to'] = validity.group(2)
    
    # Extract wind information (format like 27010KT or 27010G15KT)
    wind = re.search(r'(\d{3}|VRB)(\d{2,3})(?:G(\d{2,3}))?KT', line)
    if wind:
        data['wind_dir'] = 0 if wind.group(1) == 'VRB' else int(wind.group(1))
        data['wind_speed'] = int(wind.group(2))
        data['wind_gust'] = int(wind.group(3)) if wind.group(3) else None
    
    # Extract visibility (format like 9999 or 3000)
    vis = re.search(r'\b(\d{4})\b(?!\d)', line)
    if vis:
        data['visibility'] = int(vis.group(1))
    
    # Extract cloud information
    cloud_pattern = r'(FEW|SCT|BKN|OVC)(\d{3})(CB|TCU)?'
    clouds = re.finditer(cloud_pattern, line)
    for cloud in clouds:
        data['clouds'].append({
            'cover': cloud.group(1),
            'height': int(cloud.group(2)) * 100,
            'type': cloud.group(3) if cloud.group(3) else None
        })
    
    # Extract weather phenomena
    weather_pattern = r'\b([-+]?)(?:RA|SN|TS|FG|BR|DZ|FU|HZ|SA|DU|SQ|FC|SS|DS)\b'
    weather = re.search(weather_pattern, line)
    if weather:
        data['weather'] = weather.group(0)
        
    return data

def process_taf_file(filepath):
    """Process a TAF file and return a DataFrame with 15-minute interval predictions."""
    all_forecasts = []
    current_date = None
    
    with open(filepath, 'r', encoding='ISO-8859-1', errors='replace') as f:
        lines = f.readlines()
        
    for line in lines:
        line = line.strip()
        
        # Parse date line
        if re.match(r'\d{4}/\d{2}/\d{2}', line):
            current_date = datetime.strptime(line.split()[0], '%Y/%m/%d')
            continue
            
        # Skip empty lines or lines without TAF data
        if not line or not any(airport in line for airport in airports):
            continue
            
        # Parse TAF line
        taf_data = parse_taf_line(line)
        
        if not taf_data or not taf_data['station'] or not taf_data['valid_from'] or taf_data['station'] not in airports:
            continue

        # Convert hours >= 24 to next day(s)
        from_hour = int(taf_data['valid_from'][:2])
        from_days = from_hour // 24
        from_hour = from_hour % 24
        
        to_hour = int(taf_data['valid_to'][:2]) 
        to_days = to_hour // 24
        to_hour = to_hour % 24

        valid_from = current_date.replace(
            hour=from_hour,
            minute=int(taf_data['valid_from'][2:])
        ) + timedelta(days=from_days)

        valid_to = current_date.replace(
            hour=to_hour,
            minute=int(taf_data['valid_to'][2:])
        ) + timedelta(days=to_days)

        # Handle case where valid_to is on next day
        if valid_to < valid_from:
            valid_to += timedelta(days=1)

        # Generate 15-minute intervals
        current_time = valid_from
        while current_time <= valid_to:
            forecast = {
                'timestamp': current_time,
                'station': taf_data['station'],
                'wind_dir': taf_data['wind_dir'],
                'wind_speed': taf_data['wind_speed'],
                'wind_gust': taf_data['wind_gust'] if taf_data['wind_gust'] else 0,
                'visibility': taf_data['visibility'],
                'cloud_base': min([cloud['height'] for cloud in taf_data['clouds']]) if taf_data['clouds'] else None,
                'weather_code': hash(taf_data['weather']) % 100 if taf_data['weather'] else 0
            }
            all_forecasts.append(forecast)
            current_time += timedelta(minutes=15)
    
    if not all_forecasts:
        return pd.DataFrame()
        
    # Create DataFrame
    df = pd.DataFrame(all_forecasts)
    
    # Pivot the data so each feature is separated by airport, dropping duplicates first
    df = df.drop_duplicates(subset=['timestamp', 'station'], keep='last')
    df = df.pivot(index='timestamp', columns='station')
    
    # Flatten column names
    df.columns = [f'taf_{feat}_{stn}' for feat, stn in df.columns]
    
    # Sort by timestamp
    df.sort_index(inplace=True)
    
    # Forward fill missing values within each forecast period (6 hours)
    df = df.ffill(limit=24)  # 24 periods = 6 hours
    
    return df

# Process all TAF files
all_taf_data = []
processed_count = 0
print("Processing TAF files...")

for filepath in files:
    taf_df = process_taf_file(filepath)
    if not taf_df.empty:
        all_taf_data.append(taf_df)
        processed_count += 1
        if processed_count % 10 == 0:  # Log every 10 files
            print(f"Processed {processed_count}/{len(files)} files")

print(f"\nSuccessfully processed {processed_count} files")

# Combine all TAF data
if all_taf_data:
    print("Combining all TAF data...")
    taf_data = pd.concat(all_taf_data)
    taf_data = taf_data[~taf_data.index.duplicated(keep='last')]
    taf_data.sort_index(inplace=True)
    
    # Resample to ensure consistent 15-minute intervals
    taf_data = taf_data.resample('15min').ffill()
    print("TAF data processing complete")


In [40]:
# Sort columns and index
taf_data.fillna(0)
taf_data = taf_data[sorted(taf_data.columns)]

master_df = pd.read_csv('cleaned_data/fuser_and_metar.csv')
master_df['prediction_time'] = pd.to_datetime(master_df['prediction_time'])
master_df.set_index('prediction_time', inplace=True)

master_df = master_df.join(taf_data, how='left')
master_df = master_df.fillna(0)

master_df.to_csv('fuser_metar_taf.csv')

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def add_time_features(df):
    """Add time-based features"""
    # Convert index to datetime if needed
    df.index = pd.to_datetime(df.index)
    
    # Basic time features
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    # Peak travel times (rough approximations)
    df['is_morning_peak'] = ((df['hour'] >= 6) & (df['hour'] <= 9)).astype(int)
    df['is_evening_peak'] = ((df['hour'] >= 16) & (df['hour'] <= 19)).astype(int)
    
    return df


def add_rolling_features(df):
    """Add rolling window features"""
    windows = [4, 12, 24]  # 1 hour, 3 hours, 6 hours (assuming 15-min intervals)
    
    for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']:
        for window in windows:
            # Rolling means for actual arrivals/departures
            df[f'rolling_{window}p_arrivals_{airport}'] = df[f'actual_arrivals_{airport}'].rolling(window=window).mean()
            df[f'rolling_{window}p_departures_{airport}'] = df[f'actual_departures_{airport}'].rolling(window=window).mean()
            
            # Rolling max for weather severity
            df[f'rolling_{window}p_weather_severity_{airport}'] = df[f'weather_severity_{airport}'].rolling(window=window).max()
            
            # Rolling standard deviation for arrivals/departures
            df[f'rolling_{window}p_arrivals_std_{airport}'] = df[f'actual_arrivals_{airport}'].rolling(window=window).std()
            df[f'rolling_{window}p_departures_std_{airport}'] = df[f'actual_departures_{airport}'].rolling(window=window).std()
    
    return df

def add_lag_features(df):
    """Add lagged features"""
    lags = [1, 2, 4, 8]  # 15min, 30min, 1hr, 2hr
    
    for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']:
        for lag in lags:
            # Lag actual arrivals/departures
            df[f'lag_{lag}p_arrivals_{airport}'] = df[f'actual_arrivals_{airport}'].shift(lag)
            df[f'lag_{lag}p_departures_{airport}'] = df[f'actual_departures_{airport}'].shift(lag)
            
            # Lag weather features
            df[f'lag_{lag}p_weather_severity_{airport}'] = df[f'weather_severity_{airport}'].shift(lag)
    
    return df

def add_differential_features(df):
    """Add rate of change features"""
    for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']:
        # Rate of change for arrivals/departures
        df[f'arrivals_change_{airport}'] = df[f'actual_arrivals_{airport}'].diff()
        df[f'departures_change_{airport}'] = df[f'actual_departures_{airport}'].diff()
        
        # Rate of change for weather severity
        df[f'weather_severity_change_{airport}'] = df[f'weather_severity_{airport}'].diff()
        
        # Acceleration (second derivative) of arrivals/departures
        df[f'arrivals_acceleration_{airport}'] = df[f'arrivals_change_{airport}'].diff()
        df[f'departures_acceleration_{airport}'] = df[f'departures_change_{airport}'].diff()
    
    return df
# Add these new functions after the existing ones

def add_taf_vs_metar_features(df):
    """Add features comparing TAF forecasts with METAR actuals"""
    for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']:
        # Wind direction difference
        df[f'wind_dir_forecast_error_{airport}'] = np.abs(
            df[f'taf_wind_dir_{airport}'] - df[f'metar_wind_dir_degrees_{airport}']
        ).clip(0, 180)  # Max difference is 180 degrees
        
        # Wind speed difference
        df[f'wind_speed_forecast_error_{airport}'] = np.abs(
            df[f'taf_wind_speed_{airport}'] - df[f'metar_wind_speed_kt_{airport}']
        )
        
        # Visibility difference
        df[f'visibility_forecast_error_{airport}'] = np.abs(
            df[f'taf_visibility_{airport}'] - df[f'metar_visibility_m_{airport}']
        )
    
    return df

def add_network_features(df):
    """Add features capturing network effects between airports"""
    windows = [4, 12, 24]  # 1, 3, 6 hours
    
    # Calculate total system load
    df['total_system_arrivals'] = sum(df[f'actual_arrivals_{airport}'] 
                                    for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 
                                                  'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA'])
    df['total_system_departures'] = sum(df[f'actual_departures_{airport}'] 
                                      for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 
                                                    'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA'])
    
    # Calculate rolling system metrics
    for window in windows:
        df[f'rolling_{window}p_system_arrivals'] = df['total_system_arrivals'].rolling(window=window).mean()
        df[f'rolling_{window}p_system_departures'] = df['total_system_departures'].rolling(window=window).mean()
        
        # System-wide weather severity
        df[f'airports_with_severe_weather_{window}p'] = sum(
            (df[f'weather_severity_{airport}'] > 2).rolling(window=window).max()
            for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 
                          'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']
        )
    
    return df

def add_capacity_utilization_features(df):
    """Add features related to airport capacity utilization"""
    for airport in ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']:
        # Total operations per 15-min period
        df[f'total_operations_{airport}'] = (
            df[f'actual_arrivals_{airport}'] + df[f'actual_departures_{airport}']
        )
        
        # Rolling 1-hour operation counts
        df[f'rolling_1h_operations_{airport}'] = df[f'total_operations_{airport}'].rolling(4).sum()
        
        # Capacity pressure (ratio of current operations to rolling max)
        rolling_max = df[f'total_operations_{airport}'].rolling(96).max()  # 24-hour rolling max
        df[f'capacity_pressure_{airport}'] = (
            df[f'total_operations_{airport}'] / rolling_max.replace(0, 1)
        ).fillna(0)
    
    return df

def add_seasonal_features(df):
    """Add seasonal and cyclical features"""
    # Time of day as cyclical features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    # Day of week as cyclical features
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    # Month as cyclical features
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df

# Modify the main function to include new features
def main():
    print("Loading data...")
    df = pd.read_csv('fuser_metar_taf.csv', index_col=0, parse_dates=True)
    
    print("Adding time features...")
    df = add_time_features(df)
    
    print("Adding TAF vs METAR comparison features...")
    df = add_taf_vs_metar_features(df)
    
    print("Adding network features...")
    df = add_network_features(df)
    
    print("Adding capacity utilization features...")
    df = add_capacity_utilization_features(df)
    
    print("Adding seasonal features...")
    df = add_seasonal_features(df)
    
    print("Adding rolling features...")
    df = add_rolling_features(df)
    
    print("Adding lag features...")
    df = add_lag_features(df)
    
    print("Adding differential features...")
    df = add_differential_features(df)
    
    # Fill NaN values
    df = df.fillna(0)
    
    print("Saving enhanced dataset...")
    df.to_csv('fuser_metar_taf_enhanced.csv')
    print("Done!")

if __name__ == "__main__":
    main()

In [ ]:
import os
import glob

def get_cwam_files():
    cwam_files = []
    
    # Walk through both train and test directories
    print("Searching for CWAM files in train directory...")
    for root, dirs, files in os.walk(cwam_train):
        for file in files:
            if file.endswith('.bz2'):  # Only get the CWAM files
                cwam_files.append(os.path.join(root, file))


                
    print("\nSearching for CWAM files in test directory...")        
    for root, dirs, files in os.walk(cwam_test):
        for file in files:
            if file.endswith('.bz2'):
                cwam_files.append(os.path.join(root, file))
                
    
    return sorted(cwam_files)

cwam_files = get_cwam_files()
print(f"\nTotal CWAM files found: {len(cwam_files)}")

# Print first few files to verify
print("\nSample files:")
for file in cwam_files[:5]:
    print(file)

import h5py
import bz2
import numpy as np
from shapely.geometry import Point, Polygon
from datetime import datetime
import tempfile

def extract_timestamp(filename):
    # Extract timestamp from filename like 2022_12_18_23_45_GMT.Forecast.h5.CWAM.h5.bz2
    basename = os.path.basename(filename)
    date_part = basename.split('_GMT')[0]
    return datetime.strptime(date_part, '%Y_%m_%d_%H_%M')

def point_in_polygon(point, polygon_coords):
    """Check if point falls within polygon"""
    point = Point(point)
    polygon = Polygon(polygon_coords)
    return polygon.contains(point)

# Airport coordinates (lat, lon)
airport_coords = {
    'KATL': (33.6367, -84.4281),
    'KCLT': (35.2141, -80.9431),
    'KDEN': (39.8561, -104.6737),
    'KDFW': (32.8972, -97.0376),
    'KJFK': (40.6413, -73.7781),
    'KMEM': (35.0424, -89.9767),
    'KMIA': (25.7932, -80.2906),
    'KORD': (41.9786, -87.9048),
    'KPHX': (33.4342, -112.0117),
    'KSEA': (47.4502, -122.3088)
}

import h5py
import bz2
import numpy as np
from shapely.geometry import Point, Polygon
from datetime import datetime
import tempfile
def process_cwam_file(filepath):
    """Process single CWAM file and return features for each airport"""
    timestamp = extract_timestamp(filepath)
    features = {}
    
    with tempfile.NamedTemporaryFile() as temp_file:
        with bz2.open(filepath, 'rb') as f_in:
            temp_file.write(f_in.read())
            temp_file.flush()
            
        with h5py.File(temp_file.name, 'r') as f:
            # Process each forecast time (0 to 120 minutes, 5-minute intervals)
            forecast_times = [g for g in f['Deviation Probability'].keys() if g.startswith('FCST')]
            
            # Initialize features for each airport
            for airport in airport_coords:
                features[airport] = {
                    'timestamp': timestamp,
                    # Current conditions (FCST000)
                    'current_impact_60': 0,  # Binary: is airport currently impacted at 60% threshold?
                    'current_impact_70': 0,
                    'current_impact_80': 0,
                    'current_distance_60': float('inf'),  # Distance to nearest polygon if not impacted
                    'current_distance_70': float('inf'),
                    'current_distance_80': float('inf'),
                    # Short-term forecast (next 30 minutes)
                    'short_term_impacts': 0,  # Number of 5-min periods with impact
                    'time_to_impact': float('inf'),  # Minutes until first impact
                    # Medium-term forecast (30-120 minutes)
                    'medium_term_impacts': 0,
                    'severe_weather_approaching': 0  # Binary: weather moving towards airport
                }
            
            # Process each forecast time
            for fcst in sorted(forecast_times):
                minutes = int(fcst[4:])  # Extract minutes from FCST000
                base_path = f'Deviation Probability/{fcst}/FLVL250/Contour'
                
                for airport, coords in airport_coords.items():
                    airport_point = Point(coords)
                    
                    # Process each threshold
                    for thresh in ['060', '070', '080']:
                        thresh_path = f'{base_path}/TRSH{thresh}'
                        if thresh_path not in f:
                            continue
                            
                        group = f[thresh_path]
                        is_impacted = False
                        min_distance = float('inf')
                        
                        # Check all polygons
                        for poly_name in group.keys():
                            if not poly_name.startswith('POLY'):
                                continue
                                
                            poly_coords = group[poly_name][:]
                            poly_coords = poly_coords.reshape(-1, 2)
                            polygon = Polygon(poly_coords)
                            
                            if polygon.contains(airport_point):
                                is_impacted = True
                                min_distance = 0
                                break
                            else:
                                min_distance = min(min_distance, airport_point.distance(polygon))
                        
                        # Update features based on forecast time
                        if minutes == 0:  # Current conditions
                            features[airport][f'current_impact_{thresh[-2:]}'] = int(is_impacted)
                            features[airport][f'current_distance_{thresh[-2:]}'] = min_distance
                        elif minutes <= 30:  # Short-term forecast
                            if is_impacted:
                                features[airport]['short_term_impacts'] += 1
                                if features[airport]['time_to_impact'] == float('inf'):
                                    features[airport]['time_to_impact'] = minutes
                        else:  # Medium-term forecast
                            if is_impacted:
                                features[airport]['medium_term_impacts'] += 1
                                
                        # Check if weather is approaching airport
                        if minutes > 0 and min_distance < features[airport][f'current_distance_{thresh[-2:]}']:
                            features[airport]['severe_weather_approaching'] = 1
    
    return features

# Process files and create DataFrame
print("\nStarting CWAM file processing...")
all_features = []

for i, filepath in enumerate(cwam_files):
    try:
        if i % 10 == 0:
            print(f"\n Proccesing File {i} / {len(cwam_files)}")
        features = process_cwam_file(filepath)
        all_features.append(features)
    except Exception as e:
        print(f"Error processing {filepath}: {str(e)}")
        continue

# Convert to DataFrame with airport-specific column names
rows = []
for features in all_features:
    for airport, data in features.items():
        row = {
            'timestamp': data['timestamp'],
            f'cwam_current_impact_60_{airport}': data['current_impact_60'],
            f'cwam_current_impact_70_{airport}': data['current_impact_70'],
            f'cwam_current_impact_80_{airport}': data['current_impact_80'],
            f'cwam_distance_to_weather_60_{airport}': data['current_distance_60'],
            f'cwam_distance_to_weather_70_{airport}': data['current_distance_70'],
            f'cwam_distance_to_weather_80_{airport}': data['current_distance_80'],
            f'cwam_impacts_next_30min_{airport}': data['short_term_impacts'],
            f'cwam_minutes_until_impact_{airport}': data['time_to_impact'],
            f'cwam_impacts_30_120min_{airport}': data['medium_term_impacts'],
            f'cwam_weather_approaching_{airport}': data['severe_weather_approaching']
        }
        rows.append(row)

cwam_df = pd.DataFrame(rows)
cwam_df.set_index('timestamp', inplace=True)

# Sort columns alphabetically
cwam_df = cwam_df[sorted(cwam_df.columns)]

# Resample to 15-minute intervals and forward fill
cwam_df = cwam_df.resample('15min').ffill()

# Save to CSV
cwam_df.to_csv('cwam_features_sample.csv')
